# Como resolver problemas envolvendo Processamento de Linguagem Natural (Natural Language Processing) utilizando Aprendizado de Máquina (Machine Learning).

Dados textuais estão em toda parte e você como empresa ou alguém que esteja trabalhando para lançar um novo produto, pode utizá-los para validar, melhorar e expandir esses produtos. A ciência que extrai significado e aprendizado de dados textuais disponíveis no mundo é chamada de Processamento de Linguagem Natural (NLP).

NLP produz novos e incríveis resultados diariamente. Além disso, vem sendo estudada e praticada em grande escala. No entanto, a maior parte dos problemas, e não menos importantes, são colocados em prática com maior frequência em relação a outros:

    - Identificar diferentes grupos de usuários/clientes (ex. predizer rotatividade, tempo de vida, preferência de produtos);
    - Detectar e extrair com precisão diferentes categorias de um feedback (ex. reviews positivos e negativos, menções a   determinadas características e atributos);
    - Classificação textual de acordo com uma intenção (ex. se alguém está requisitando ajuda, se está tendo um problema urgente, se uma notícia é falsa ou verdadedaira).
    
Mesmo existindo vários artigos e tutoriais online sobre o tema, é difícil encontrar material e dicas de como tratar problemas que envolvem NLP, de forma eficiente, partindo do zero. Sendo assim, este guia irá auxiliar na construção de soluções que envolvem Aprendizado de Máquina (ML). Depois de acomnpanhar estas orientações, será possível:

    - Coletar, preparar e inspecionar dados;
    - Construir modelos preditivos simples;
    - Interpretar e entender seus modelos, certinfcando-se que ele está aprendendo informações relevantes e não ruído.

Este guia foi construido partindo de uma tradução livre do workshoop disponibilizado por Emmanuel Ameisen em:

https://blog.insightdatascience.com/how-to-solve-90-of-nlp-problems-a-step-by-step-guide-fda605278e4e
https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb

# Passo 1: Coletando dados

Todo problema em ML começa com dados, como uma lista de e-mails, postagens em redes sociais ou chats. Algumas fontes são comuns:

    - Revisão de produtos (Amazon, Yelp e App Stores);
    - Contepudo gerado por usuários (Tweets, Facebook e StackOverflow);
    - Solução de problemas (solicitação de clientes, abertura de chamados, informações de bate-papo).
    
Os dados utilizados neste guia, são dados disponibilizados por Figure Eight, chamados de "Disasters on Social Media". Esse conjunto de dados possui mais de 10.000 tweets que foram selecionados a partir de buscas envolvendo chaves como "em chamas", "quarentena" e "pandemônio", relacionando posteriormente o resultado dessas buscas com eventos realmente desastrosos e não-desastrosos.

A tarefa a ser realizada nestes dados é identificar um tweet realmente relevante em relação a um desastre. Uma aplicação em potêncial seria avisar aos bombeiros sobre um desastre que realmente está acontecendo de forma rapidamente, mas contornando uma publicação referente a um filme de ação, que possuem as mesmas palavras chaves.

A seguir a importação das bibliotecas úteis para o desenvolvimento do passo a passo.

In [50]:
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs

# Limpando os dados

A primeira regra é saber examinar os dados e depois limpá-los. Um conjunmto de dados limpo permitirá que um modelo aprenda apenas recusros signifivamente relevantes e não se adeque a ruídos. Ruídos são características do texto que não são significativos para caracterizar o que se pretende avaliar. No exemplo dos tweets, a hashtag (#) usada para compor determinadas palavras em nada caracteriza o texto propriamente dito. Outras caracterpisticas que precisam ser consideradas na limpeza dos dados são:

    - Remover caracteteres irrelevantes como os não alfanuméricos;
    - Quebrar o seu texto em termos;
    - Remover palavras que não são relevantes, como as menções indicadas com "@";
    - Converter termos para minúsculos, homogeneizando os dados (OLÁ - olá);
    - Combinar palavras escritas incorretamente para uma única representação (legaaalll - legal);
    - Considerar lematização (reduzir palavras como "sou", "é" para uma forma desflexionada como "ser").

Após considerar as opções de limpeza acima, é possível começar a usar os dados para treinar um modelo de predição.

A seguir o código de limpeza dos dados que serão utilizados para treinar o modelo.

In [51]:
# Arquivo de entrada com os dados rotulados
input_file = codecs.open('socialmedia_relevant_cols.csv', 'r', encoding='utf-8', errors='replace')
#Arquivo de saída com os dados rotulados e limpos a serem salvos
output_file = open('socialmedia_relevant_cols_clean.csv', 'w', encoding='utf-8')

#Copiando dados de entrada para os dados de saída
def sanitize_characters(raw, clean):    
    for line in input_file:
        out = line
        output_file.write(line)
sanitize_characters(input_file, output_file)

In [52]:
# Inspecionando os dados
questions = pd.read_csv("socialmedia_relevant_cols_clean.csv")
questions.columns=['text', 'choose_one', 'class_label']

Dados sujos:

In [53]:
#Primeiras linhas
questions.head()

,text,choose_one,class_label
0,Just happened a terrible car crash,Relevant,1
1,Our Deeds are the Reason of this #earthquake M...,Relevant,1
2,"Heard about #earthquake is different cities, s...",Relevant,1
3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,Forest fire near La Ronge Sask. Canada,Relevant,1


In [54]:
#Últimas linhas
questions.tail()

,text,choose_one,class_label
10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,Relevant,1
10872,Police investigating after an e-bike collided ...,Relevant,1
10873,The Latest: More Homes Razed by Northern Calif...,Relevant,1
10874,MEG issues Hazardous Weather Outlook (HWO) htt...,Relevant,1
10875,#CityofCalgary has activated its Municipal Eme...,Relevant,1


In [55]:
# Estatística descritiva dos dados
questions.describe()

,class_label
count,10876.000000
mean,0.432604
std,0.498420
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,2.000000


In [56]:
# Expressões regulares para limpeza dos dados
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [57]:
#Limpeza e regravação do arquivo de saída
questions = standardize_text(questions, "text")

questions.to_csv("clean_data.csv")

clean_questions = pd.read_csv("clean_data.csv")

Dados limpos:

In [58]:
#Primeiras linhas
clean_questions.head()

,Unnamed: 0,text,choose_one,class_label
0,0,just happened a terrible car crash,Relevant,1
1,1,our deeds are the reason of this earthquake m...,Relevant,1
2,2,"heard about earthquake is different cities, s...",Relevant,1
3,3,"there is a forest fire at spot pond, geese are...",Relevant,1
4,4,forest fire near la ronge sask canada,Relevant,1


In [59]:
#Últimas linhas
clean_questions.tail()

,Unnamed: 0,text,choose_one,class_label
10871,10871,m1 94 01 04 utc ?5km s of volcano hawaii,Relevant,1
10872,10872,police investigating after an e bike collided ...,Relevant,1
10873,10873,the latest more homes razed by northern calif...,Relevant,1
10874,10874,meg issues hazardous weather outlook (hwo),Relevant,1
10875,10875,cityofcalgary has activated its municipal eme...,Relevant,1


Distribuição das classes:

In [60]:
clean_questions.groupby("class_label").count()

,Unnamed: 0,text,choose_one
class_label,,,
0,6187,6187,6187
1,4673,4673,4673
2,16,16,16


É possível obervar que os dados são balanceados, com uma leve elevação da classe "Not Relevante".

# Quebrando os dados

Agora que temos os dados limpos, vamos transformá-los para que o modelo possa entender. Logo:

    - Quebrar as sentenças em listas de palavras separadas;
    - Dividir os dados para treinamento e teste do modelo.

In [64]:
from nltk.tokenize import RegexpTokenizer

# Método de quebra dos dados
tokenizer = RegexpTokenizer(r'\w+')

# Gerando listas de sentenças quebradas
clean_questions["tokens"] = clean_questions["text"].apply(tokenizer.tokenize)

In [65]:
#Primeiras linhas
clean_questions.head()

,Unnamed: 0,text,choose_one,class_label,tokens
0,0,just happened a terrible car crash,Relevant,1,"[just, happened, a, terrible, car, crash]"
1,1,our deeds are the reason of this earthquake m...,Relevant,1,"[our, deeds, are, the, reason, of, this, earth..."
2,2,"heard about earthquake is different cities, s...",Relevant,1,"[heard, about, earthquake, is, different, citi..."
3,3,"there is a forest fire at spot pond, geese are...",Relevant,1,"[there, is, a, forest, fire, at, spot, pond, g..."
4,4,forest fire near la ronge sask canada,Relevant,1,"[forest, fire, near, la, ronge, sask, canada]"


In [66]:
#Últimas linhas
clean_questions.tail()

,Unnamed: 0,text,choose_one,class_label,tokens
10871,10871,m1 94 01 04 utc ?5km s of volcano hawaii,Relevant,1,"[m1, 94, 01, 04, utc, 5km, s, of, volcano, haw..."
10872,10872,police investigating after an e bike collided ...,Relevant,1,"[police, investigating, after, an, e, bike, co..."
10873,10873,the latest more homes razed by northern calif...,Relevant,1,"[the, latest, more, homes, razed, by, northern..."
10874,10874,meg issues hazardous weather outlook (hwo),Relevant,1,"[meg, issues, hazardous, weather, outlook, hwo]"
10875,10875,cityofcalgary has activated its municipal eme...,Relevant,1,"[cityofcalgary, has, activated, its, municipal..."
